# Код для обработки таблицы после degust + edgeR

#### Результат каждого из экспериментов записан в отдельных файлах после обработки в degust. Соединяем все эксперименты в одну таблицу, после чего сопоставляем данные из annotation.

In [1]:
import pandas as pd

#### Считаем все данные таблицы

In [2]:
experiment_1 = pd.read_csv("Experiment_1.csv")
experiment_2 = pd.read_csv('Experiment_2.csv')
experiment_3 = pd.read_csv('Experiment_3.csv')
Annotation = pd.read_csv('mc2_feature_table.txt', sep = '\t')

#### Соединяем данные экспериментов в одну таблицу degust_out. Далее сопоставляем данные с annotation. Сортируем данные в полученной таблице по "locus tag". Далее удаляем ненужные нам колонки и расставляем оставшиеся в удобном для работы порядке

In [3]:
Annotation['id_'] = Annotation['genomic_accession'] + '_cds_' + Annotation['product_accession']
cols = ['control0_2_x', 'control0_3_x', 'exp1_1', 'exp1_2',
       'exp1_3', 'control0_1 CPM_x', 'control0_2 CPM_x', 'control0_3 CPM_x',
       'exp1_1 CPM', 'exp1_2 CPM', 'exp1_3 CPM','control0_1_y', 'control0_2_y',
       'control0_3_y', 'exp2_1', 'exp2_2', 'exp2_3', 'control0_1 CPM_y',
       'control0_2 CPM_y', 'control0_3 CPM_y', 'exp2_1 CPM', 'exp2_2 CPM',
       'exp2_3 CPM','control0_1', 'control0_2', 'control0_3', 'exp3_1', 'exp3_2', 'exp3_3',
       'control0_1 CPM', 'control0_2 CPM', 'control0_3 CPM', 'exp3_1 CPM',
       'exp3_2 CPM', 'exp3_3 CPM']

In [4]:
def merge_tables(exp_1, exp_2, exp_3, annotation):
    # Соединяем данные экспериментов в одну таблицу degust_out.
    merge_1_2 = exp_1.merge(exp_2, left_on = 'target_id',right_on = 'target_id' , how = 'inner')
    degust_out = merge_1_2.merge(exp_3, left_on = 'target_id',right_on = 'target_id' , how = 'inner')
    df = degust_out
    df['id_']= df['target_id'].apply(lambda x: '_'.join(x.split('|')[1:]))
    df['id_']= df['id_'].apply(lambda x: '_'.join(x.split('_')[:-1]))
    # Далее сопоставляем данные с annotation.
    degust_and_annotation_merged = df.merge(annotation, left_on = 'id_',right_on = 'id_' , how = 'inner')
    #Сортируем данные в полученной таблице по "locus tag".
    new_table = degust_and_annotation_merged.sort_values(by='locus_tag')
    # Далее удаляем ненужные нам колонки и расставляем оставшиеся в удобном для работы порядке
    del new_table['control0_1_x']
    for i in cols:
        del new_table[i]
    col = new_table.columns.tolist()
    col_1 = col[:33] + col[34:]
    col_1[35] = col[33]
    col_1 = col_1[-1:] + col_1[:-1]
    new_table = new_table[col_1]
    return(new_table)

#### На этом этапе можно сохранить получившуюся таблицу, в которой содержится информация по всем экспериментам, полученная из degust, а также некоторая информация из annotation

In [6]:
def save(new_table):
    new_table.to_csv('new_table')

## Далее будет производиться поиск нужных генов по некоторым условиям: P-value и FDR <0.05, Fold Change (FC) > 2 при оверэкспресси и < -2 при андэрэкспрессии, интересующие нас гены должны встречаться не поодиночке, а кластерами

In [7]:
def filter_table(new_table):
    appropriate_results = new_table.loc[(new_table['P value'] < 0.05) & (new_table['P value_x'] < 0.05) & (new_table['P value_y'] < 0.05) & (new_table['FDR'] < 0.05) & (new_table['FDR_x'] < 0.05) & (new_table['FDR_y'] < 0.05) & (abs(new_table['exp1']) > 2)& (abs(new_table['exp2']) > 2) & (abs(new_table['exp3']) > 2)]
    return(appropriate_results)

In [8]:
def underexpression(appropriate_results):
    under_expression = appropriate_results.loc[(appropriate_results['exp1'] < 0) & (appropriate_results['exp2'] < 0) &(appropriate_results['exp3'] < 0)]
    return(under_expression)

In [9]:
def clustered_under(appropriate_results):
    under_expression = appropriate_results.loc[(appropriate_results['exp1'] < 0) & (appropriate_results['exp2'] < 0) &(appropriate_results['exp3'] < 0)]
    ind_under = list(under_expression.locus_tag.index)
    cluster_under = set()
    for i in range(1,len(ind_under)-1):
        if (int(under_expression.locus_tag[ind_under[i+1]][-4:]) - int(under_expression.locus_tag[ind_under[i]][-4:]) == 1):
            cluster_under.add(under_expression.locus_tag[ind_under[i]])
            cluster_under.add(under_expression.locus_tag[ind_under[i+1]])
    return(cluster_under)

In [10]:
def overexpession(appropriate_results):
    over_expression = appropriate_results.loc[(appropriate_results['exp1'] > 0) & (appropriate_results['exp2'] > 0) &(appropriate_results['exp3'] > 0)]
    return(over_expression)

In [11]:
def clustered_over(appropriate_results):
    over_expression = appropriate_results.loc[(appropriate_results['exp1'] > 0) & (appropriate_results['exp2'] > 0) &(appropriate_results['exp3'] > 0)]
    ind_over = list(over_expression.locus_tag.index)
    cluster_over = set()
    for i in range(1,len(ind_over)-1):
        if (int(over_expression.locus_tag[ind_over[i+1]][-4:]) - int(over_expression.locus_tag[ind_over[i]][-4:]) == 1):
            cluster_over.add(over_expression.locus_tag[ind_over[i]])
            cluster_over.add(over_expression.locus_tag[ind_over[i+1]])
    return(cluster_over)

## Далее будет приведен пример использования кода. 

#### Используемые функции: merge_tables(), save(), filter_table(), underexpression(), overexpession(). Вводятся новые переменные My_new_table и My_filtered_table по аналогии с new_table и appropriate_results

In [15]:
Result = pd.read_csv("RESULT.tsv")

ParserError: Error tokenizing data. C error: Expected 13 fields in line 31, saw 14


In [83]:
save(My_new_table)

In [84]:
My_filtered_table = filter_table(My_new_table)

In [85]:
My_filtered_table

,locus_tag,target_id,control0_x,exp1,FDR_x,AveExpr_x,P value_x,control0_y,exp2,FDR_y,...,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,feature_interval_length,product_length
1357,MSMEG_0018,lcl|NC_008596.1_cds_YP_884438.1_15,0,2.374681,1.971947e-11,3.008629,3.986757e-12,0,5.658576,5.510336e-66,...,19018,+,YP_884438.1,WP_011726610.1,NaN,ABC transporter permease,NaN,4534027,1770,589.0
969,MSMEG_0105,lcl|NC_008596.1_cds_YP_884521.1_98,0,-2.684380,1.260959e-17,3.288435,1.820947e-18,0,-2.517641,1.536128e-14,...,128734,+,YP_884521.1,WP_011726671.1,NaN,LuxR family transcriptional regulator,NaN,4536254,651,216.0
622,MSMEG_0113,lcl|NC_008596.1_cds_YP_884529.1_106,0,-4.115643,1.551499e-28,3.080067,1.439011e-29,0,-3.834965,7.325003e-24,...,137810,+,YP_884529.1,WP_011726675.1,NaN,taurine transporter permease TauC,NaN,4532372,861,286.0
245,MSMEG_0114,lcl|NC_008596.1_cds_YP_884530.1_107,0,-3.917611,3.350661e-72,4.976714,1.227129e-73,0,-3.489585,8.852760e-57,...,138845,+,YP_884530.1,WP_011726676.1,NaN,extracellular solute-binding protein,NaN,4533591,1029,342.0
699,MSMEG_0116,lcl|NC_008596.1_cds_YP_884531.1_108,0,-2.713073,1.067028e-25,3.912504,1.111984e-26,0,-2.991794,3.820092e-28,...,139626,+,YP_884531.1,WP_003891448.1,NaN,taurine import ATP-binding protein TauB,NaN,4533525,798,265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,MSMEG_6790,lcl|NC_008596.1_cds_YP_890998.1_6576,0,-3.856302,8.295504e-72,4.907769,3.050453e-73,0,-4.128618,1.993124e-66,...,6838949,+,YP_890998.1,WP_003898170.1,NaN,"AP endonuclease, family protein 2",NaN,4535020,921,306.0
213,MSMEG_6791,lcl|NC_008596.1_cds_YP_890999.1_6577,0,-3.494764,6.488138e-84,5.329479,2.067086e-85,0,-3.166871,7.129986e-57,...,6839937,+,YP_890999.1,WP_003898171.1,NaN,3-hydroxybutyryl-CoA dehydrogenase,NaN,4533563,963,320.0
192,MSMEG_6792,lcl|NC_008596.1_cds_YP_891000.1_6578,0,-3.000433,1.845720e-93,5.726642,5.303318e-95,0,-2.531607,3.115459e-64,...,6841405,+,YP_891000.1,WP_003898172.1,NaN,inner membrane permease YgbN,NaN,4532005,1401,466.0
172,MSMEG_6826,lcl|NC_008596.1_cds_YP_891030.1_6608,0,-3.293967,3.740448e-101,5.532583,9.633727e-103,0,-3.519697,4.164285e-83,...,6877969,+,YP_891030.1,WP_011731564.1,NaN,L-lactate permease,NaN,4534785,1683,560.0


In [86]:
underexpression(My_filtered_table)

,locus_tag,target_id,control0_x,exp1,FDR_x,AveExpr_x,P value_x,control0_y,exp2,FDR_y,...,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,feature_interval_length,product_length
969,MSMEG_0105,lcl|NC_008596.1_cds_YP_884521.1_98,0,-2.684380,1.260959e-17,3.288435,1.820947e-18,0,-2.517641,1.536128e-14,...,128734,+,YP_884521.1,WP_011726671.1,NaN,LuxR family transcriptional regulator,NaN,4536254,651,216.0
622,MSMEG_0113,lcl|NC_008596.1_cds_YP_884529.1_106,0,-4.115643,1.551499e-28,3.080067,1.439011e-29,0,-3.834965,7.325003e-24,...,137810,+,YP_884529.1,WP_011726675.1,NaN,taurine transporter permease TauC,NaN,4532372,861,286.0
245,MSMEG_0114,lcl|NC_008596.1_cds_YP_884530.1_107,0,-3.917611,3.350661e-72,4.976714,1.227129e-73,0,-3.489585,8.852760e-57,...,138845,+,YP_884530.1,WP_011726676.1,NaN,extracellular solute-binding protein,NaN,4533591,1029,342.0
699,MSMEG_0116,lcl|NC_008596.1_cds_YP_884531.1_108,0,-2.713073,1.067028e-25,3.912504,1.111984e-26,0,-2.991794,3.820092e-28,...,139626,+,YP_884531.1,WP_003891448.1,NaN,taurine import ATP-binding protein TauB,NaN,4533525,798,265.0
228,MSMEG_0130,lcl|NC_008596.1_cds_YP_884546.1_123,0,-2.508513,2.054934e-78,6.351352,7.005805e-80,0,-2.786720,5.639270e-88,...,151123,-,YP_884546.1,WP_011726687.1,NaN,GntR family transcriptional regulator,NaN,4535047,693,230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,MSMEG_6790,lcl|NC_008596.1_cds_YP_890998.1_6576,0,-3.856302,8.295504e-72,4.907769,3.050453e-73,0,-4.128618,1.993124e-66,...,6838949,+,YP_890998.1,WP_003898170.1,NaN,"AP endonuclease, family protein 2",NaN,4535020,921,306.0
213,MSMEG_6791,lcl|NC_008596.1_cds_YP_890999.1_6577,0,-3.494764,6.488138e-84,5.329479,2.067086e-85,0,-3.166871,7.129986e-57,...,6839937,+,YP_890999.1,WP_003898171.1,NaN,3-hydroxybutyryl-CoA dehydrogenase,NaN,4533563,963,320.0
192,MSMEG_6792,lcl|NC_008596.1_cds_YP_891000.1_6578,0,-3.000433,1.845720e-93,5.726642,5.303318e-95,0,-2.531607,3.115459e-64,...,6841405,+,YP_891000.1,WP_003898172.1,NaN,inner membrane permease YgbN,NaN,4532005,1401,466.0
172,MSMEG_6826,lcl|NC_008596.1_cds_YP_891030.1_6608,0,-3.293967,3.740448e-101,5.532583,9.633727e-103,0,-3.519697,4.164285e-83,...,6877969,+,YP_891030.1,WP_011731564.1,NaN,L-lactate permease,NaN,4534785,1683,560.0


In [87]:
overexpession(My_filtered_table)

,locus_tag,target_id,control0_x,exp1,FDR_x,AveExpr_x,P value_x,control0_y,exp2,FDR_y,...,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,feature_interval_length,product_length
1357,MSMEG_0018,lcl|NC_008596.1_cds_YP_884438.1_15,0,2.374681,1.971947e-11,3.008629,3.986757e-12,0,5.658576,5.510336e-66,...,19018,+,YP_884438.1,WP_011726610.1,NaN,ABC transporter permease,NaN,4534027,1770,589.0
152,MSMEG_0172,lcl|NC_008596.1_cds_YP_884588.1_165,0,3.375515,2.229605e-116,7.856242,5.078600e-118,0,3.980267,1.732977e-211,...,198190,-,YP_884588.1,WP_011726711.1,NaN,transmembrane protein,NaN,4533695,603,200.0
417,MSMEG_0183,lcl|NC_008596.1_cds_YP_884600.1_177,0,3.081647,3.524454e-44,4.501809,2.193273e-45,0,3.320937,1.130317e-61,...,208584,-,YP_884600.1,WP_011726722.1,NaN,hypothetical protein,NaN,4535626,552,183.0
69,MSMEG_0185,lcl|NC_008596.1_cds_YP_884601.1_178,0,3.884675,6.597917e-217,6.845022,6.875900e-219,0,4.019766,5.523501e-206,...,211442,-,YP_884601.1,WP_011726723.1,NaN,MmpL6 protein,NaN,4537140,2862,953.0
408,MSMEG_0186,lcl|NC_008596.1_cds_YP_884602.1_179,0,3.179179,4.466879e-45,4.193491,2.719895e-46,0,3.293628,2.148194e-45,...,211915,-,YP_884602.1,WP_011726724.1,NaN,MmpS2 protein,NaN,4536136,495,164.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,MSMEG_6566,lcl|NC_008596.1_cds_YP_890778.1_6356,0,5.176846,3.179009e-50,3.613272,1.732198e-51,0,5.665122,4.447291e-67,...,6621258,-,YP_890778.1,WP_003897985.1,NaN,hypothetical protein,NaN,4531126,156,51.0
1864,MSMEG_6603,lcl|NC_008596.1_cds_YP_890814.1_6392,0,2.892331,1.463046e-07,1.670288,4.062203e-08,0,3.679483,4.726630e-16,...,6659215,-,YP_890814.1,WP_011731395.1,NaN,NADH:flavin oxidoreductase,NaN,4533716,396,131.0
650,MSMEG_6696,lcl|NC_008596.1_cds_YP_890904.1_6482,0,9.101465,9.687353e-28,3.340549,9.388815e-29,0,6.417079,9.861619e-04,...,6747487,-,YP_890904.1,WP_011731467.1,NaN,hypothetical protein,NaN,4533444,405,134.0
40,MSMEG_6747,lcl|NC_008596.1_cds_YP_890955.1_6533,0,5.225258,3.448134e-319,7.736355,2.104720e-321,0,5.361188,1.030680e-279,...,6793198,+,YP_890955.1,WP_003898125.1,NaN,oxidoreductase,NaN,4531228,918,305.0


In [94]:
clustered_over(My_filtered_table)

{'MSMEG_0185',
 'MSMEG_0186',
 'MSMEG_0493',
 'MSMEG_0494',
 'MSMEG_0573',
 'MSMEG_0574',
 'MSMEG_0575',
 'MSMEG_0576',
 'MSMEG_0601',
 'MSMEG_0602',
 'MSMEG_0616',
 'MSMEG_0617',
 'MSMEG_0618',
 'MSMEG_0619',
 'MSMEG_0620',
 'MSMEG_0621',
 'MSMEG_1381',
 'MSMEG_1382',
 'MSMEG_1748',
 'MSMEG_1749',
 'MSMEG_1750',
 'MSMEG_1770',
 'MSMEG_1771',
 'MSMEG_1950',
 'MSMEG_1951',
 'MSMEG_2067',
 'MSMEG_2068',
 'MSMEG_2069',
 'MSMEG_2130',
 'MSMEG_2131',
 'MSMEG_2905',
 'MSMEG_2906',
 'MSMEG_3384',
 'MSMEG_3385',
 'MSMEG_3583',
 'MSMEG_3584',
 'MSMEG_3585',
 'MSMEG_3598',
 'MSMEG_3599',
 'MSMEG_3600',
 'MSMEG_3601',
 'MSMEG_3602',
 'MSMEG_3603',
 'MSMEG_3604',
 'MSMEG_4063',
 'MSMEG_4064',
 'MSMEG_4509',
 'MSMEG_4510',
 'MSMEG_4511',
 'MSMEG_4512',
 'MSMEG_4513',
 'MSMEG_4514',
 'MSMEG_4515',
 'MSMEG_4890',
 'MSMEG_4891',
 'MSMEG_5343',
 'MSMEG_5344',
 'MSMEG_5558',
 'MSMEG_5559',
 'MSMEG_5659',
 'MSMEG_5660',
 'MSMEG_5661',
 'MSMEG_5739',
 'MSMEG_5740',
 'MSMEG_5819',
 'MSMEG_5820',
 'MSMEG_58